In [73]:
import pandas as pd 
import os
import copy
import statsmodels.api as sm
import numpy as np
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error,root_mean_squared_error
from sklearn.metrics import r2_score
import optuna
import seaborn as sns

In [74]:
import pandas as pd
import os

# Define the path to the folder containing the CSV files
folder_path = r"C:\Users\aadit\ESA Research\Contract Pricing"

# List all files in the directory
file_names = [
    'Contracts Pricing - December 24.csv',
    'Contracts Pricing - December 25.csv',
    'Contracts Pricing - March 25.csv',
    'Contracts Pricing - March 26.csv',
    'Contracts Pricing - May 25.csv',
    'Contracts Pricing - May 26.csv',
    'Contracts Pricing - July 24.csv',
    'Contracts Pricing - July 25.csv',
    'Contracts Pricing - September 24.csv',
    'Contracts Pricing - September 25.csv'
]

# Create an empty dictionary to store DataFrames
dataframes = {}

# Loop through each file and read it into a DataFrame
for file_name in file_names:
    file_path = os.path.join(folder_path, file_name)
    df_name = file_name.split('.')[0]  # Use the file name without extension as the DataFrame name
    dataframes[df_name] = pd.read_csv(file_path)

In [75]:
import copy
import pandas as pd
import numpy as np

dfs_list = list(dataframes.values())

for df in dfs_list:
    # Convert the 'Date' column to datetime with the correct format
    df['Date'] = pd.to_datetime(df['Date'], format="%b %d, %Y")
    df['Maturity Date'] = pd.to_datetime(df['Maturity Date'], format="%m/%d/%Y")

    
    # Replace '-' with NaN in 'Adj Close' column
    df['Adj Close'].replace('-', np.nan, inplace=True)
    
    # Replace '-' with 0 in 'Volume' column
    df['Volume'].replace('-', 0, inplace=True)
    
    # Drop rows with NaN values in 'Adj Close' column
    df.dropna(subset=['Adj Close'], inplace=True)
    nan_values = df.isna().sum()
    df['Time to Maturity'] = (df['Maturity Date'] - df['Date']).dt.days
    

    df['Adj Close'] = pd.to_numeric(df['Adj Close'])
    # Replace commas with empty string in 'Volume' column
    df['Volume'] = df['Volume'].str.replace(',', '')

# Convert 'Volume' column to numeric type
    df['Volume'] = pd.to_numeric(df['Volume'])
    df['Maturity Month'] = pd.to_numeric(df['Maturity Month'])
    df['Maturity Year'] = pd.to_numeric(df['Maturity Year'])






In [76]:
for df in dfs_list:
    df.iloc[:] = df.iloc[::-1].values
    df['Returns'] = df['Adj Close'].pct_change() * 100
    df['Next Day Returns'] = df['Returns'].shift(-1)
    df = df.iloc[1::-1]

In [77]:
merged_df = pd.concat(dfs_list)
print(type(merged_df))

<class 'pandas.core.frame.DataFrame'>


In [78]:

def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 5, 30),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None])
    }

    rf_model = RandomForestRegressor(**params, random_state=42)

    # Train the model
    rf_model.fit(X_train, y_train)

    # Evaluate the model
    y_pred = rf_model.predict(X_test)
    return mean_squared_error(y_test, y_pred)

In [80]:
merged_df = merged_df.dropna()

x_vars = ['Returns', 'Maturity Month', 'Maturity Year', 'Time to Maturity', 'Volume']
X = merged_df.loc[:, x_vars]
y = merged_df['Next Day Returns']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Fit the OLS model
X_train = sm.add_constant(X_train)
model = sm.OLS(y_train, X_train).fit()

# Make predictions on the test set
X_test = sm.add_constant(X_test)
y_pred = model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))  # Corrected function name

# Print model summary and evaluation metrics
print(model.summary())
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")


                            OLS Regression Results                            
Dep. Variable:       Next Day Returns   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.528
Date:                Thu, 30 May 2024   Prob (F-statistic):              0.178
Time:                        19:06:22   Log-Likelihood:                -3256.8
No. Observations:                2440   AIC:                             6526.
Df Residuals:                    2434   BIC:                             6560.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const                0.9323      0.956  

In [81]:
# Set up Optuna study
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

# Get the best hyperparameters
best_params = study.best_params

# Train the model with the best hyperparameters
rf_best = RandomForestRegressor(**best_params, random_state=42)
rf_best.fit(X_train, y_train)

# Evaluate the model
y_pred = rf_best.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)

print(f"Best Hyperparameters: {best_params}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")


[I 2024-05-30 19:07:17,962] A new study created in memory with name: no-name-2a188ba2-a3dd-4731-a8a9-a0c0173afa2a
[I 2024-05-30 19:07:26,352] Trial 0 finished with value: 0.9750148439917656 and parameters: {'n_estimators': 568, 'max_depth': 12, 'min_samples_split': 14, 'min_samples_leaf': 4, 'max_features': None}. Best is trial 0 with value: 0.9750148439917656.
[I 2024-05-30 19:07:27,423] Trial 1 finished with value: 0.9724691983497477 and parameters: {'n_estimators': 271, 'max_depth': 21, 'min_samples_split': 17, 'min_samples_leaf': 10, 'max_features': 'log2'}. Best is trial 1 with value: 0.9724691983497477.
[I 2024-05-30 19:07:28,752] Trial 2 finished with value: 0.9569873039929422 and parameters: {'n_estimators': 593, 'max_depth': 5, 'min_samples_split': 16, 'min_samples_leaf': 1, 'max_features': 'log2'}. Best is trial 2 with value: 0.9569873039929422.
[I 2024-05-30 19:07:32,064] Trial 3 finished with value: 0.9907400029546503 and parameters: {'n_estimators': 902, 'max_depth': 19, '

Best Hyperparameters: {'n_estimators': 156, 'max_depth': 6, 'min_samples_split': 19, 'min_samples_leaf': 3, 'max_features': None}
Mean Absolute Error (MAE): 0.70
Root Mean Squared Error (RMSE): 0.98


c:\Users\aadit\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
